In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, config.root_path)

In [3]:
import tensorflow as tf

from tensorflow.python import keras
import json
import pandas as pd
import numpy as np
import os

from src.encoders.context_encoder_ldabert_2 import ContextEncoderSimple, ContextEncoderComplex
from src.dataset.ldabert_2 import LDABERT2Dataset

## LDA BERT 2 Dataset

In [13]:
dataset = LDABERT2Dataset(dataset_type="clinical",
                       pct_data=0.001,
                          max_seq_length=128,
                       max_segment_length=5,
                       augment_pct=0.001,
                         split="train")

In [14]:
sentences, tokenized_sentences, labels = dataset.process()

artificial segments False
artificial segments False


In [15]:
lda_sentences, lda_token_lists, lda_new_labels = dataset.preprocess_lda(sentences, labels)

Preprocessing raw texts ...
sentences length 10
Preprocessing raw texts. Done!
lda sentences length 10


## Tri Encoder

In [21]:
context_encoder = ContextEncoder(final_dropout=0.5, dense_neurons=64, max_sentence_length=128, gamma=15)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [22]:
# test_data = [['This is an example sentence', 'This is a new sentence', 'And this is a continuation'], 
#              ['This is a new sentence', 'And this is a continuation', 'And another continuation'], 
#              ['And this is a continuation', 'And another continuation', 'And yet another continuation']]

In [25]:
context_encoder([[left_input, mid_input, right_input, lda_left_input, lda_mid_input, lda_right_input]])

IndexError: list index out of range

In [382]:
context_encoder.summary()

Model: "context_encoder_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_bert_model_24 (TFBertMode multiple                  109482240 
_________________________________________________________________
dense_input_left (Dense)     multiple                  49856     
_________________________________________________________________
dense_input_mid (Dense)      multiple                  49856     
_________________________________________________________________
dense_input_right (Dense)    multiple                  49856     
_________________________________________________________________
dense_output (Dense)         multiple                  193       
_________________________________________________________________
final_dropout (Dropout)      multiple                  0         
Total params: 109,632,001
Trainable params: 109,632,001
Non-trainable params: 0
__________________________________

## Generate Vectors

In [46]:
dataset_types = ["clinical", "wiki", "fiction"]
dataset_splits = ["train", "test"]

for d in dataset_types:
    for split in dataset_splits:
        dataset = LDABERT2Dataset(dataset_type=d,
                               pct_data=1,
                               max_segment_length=5,
                               augment_pct=1,
                             split=split)
 
        sentences, tokenized_sentences, labels = dataset.process()

        # vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
        vectors_filename = '{}_{}.pkl'.format(dataset.pct_data, dataset.augment_pct)

        saved_vectors, saved_tokens, saved_labels, saved_tokenized_sentences = dataset.get_saved_vectors(split, dataset.dataset_type, vectors_filename)

        if len(saved_vectors) == 0:
            saved_vectors, saved_tokens, saved_labels, saved_tokenized_sentences = dataset.create_vectors(split, dataset.dataset_type, vectors_filename)

something went wrong [Errno 2] No such file or directory: 'C:\\Users\\Computer\\Google Drive\\SCHOOL\\PhD\\Code\\context-encoder-v2\\data\\lda_bert_2\\generated_vectors\\train\\clinical\\1_1.pkl'
root path C:\Users\Computer\Google Drive\SCHOOL\PhD\Code\context-encoder-v2
Preprocessing raw texts ...
sentences length 15051
Preprocessing raw texts. Done!
lda sentences length 15051
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
saving vectors... 15051 15051 15051
something went wrong [Errno 2] No such file or directory: 'C:\\Users\\Computer\\Google Drive\\SCHOOL\\PhD\\Code\\context-encoder-v2\\data\\lda_bert_2\\generated_vectors\\test\\clinical\\1_1.pkl'
root path C:\Users\Computer\Google Drive\SCHOOL\PhD\Code\context-encoder-v2
Preprocessing raw texts ...
sentences length 1071
Preprocessing raw texts. Done!
lda sentences length 1071
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
saving vectors... 1071 1071

In [9]:
dataset = LDABERTDataset(dataset_type="clinical",
                       pct_data=1,
                       max_segment_length=5,
                       augment_pct=1)

In [10]:
sentences, tokenized_sentences, labels = dataset.process(preprocess=False)

In [11]:
len(sentences)

16122

In [12]:
tokenized_sentences[:5]

[]

### Vectors

In [16]:
config.root_path

'G:\\My Drive\\SCHOOL\\PhD\\Code\\context-encoder-v2'

In [5]:
# vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
vectors_filename = '{}_{}.pkl'.format(dataset.pct_data, dataset.augment_pct)

saved_vectors, saved_tokens, saved_labels, saved_tokenized_sentences = dataset.get_saved_vectors("train", dataset.dataset_type, vectors_filename)

if len(saved_vectors) == 0:
    saved_vectors, saved_tokens, saved_labels, saved_tokenized_sentences = dataset.create_vectors("train", dataset.dataset_type, vectors_filename)

In [6]:
left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

In [7]:
left_input

{'input_ids': <tf.Tensor: shape=(10, 55), dtype=int32, numpy=
 array([[  101,  2065,  2017,  2342,  2393,  2007,  2115,  5544,  1010,
          3531,  2123,  1005,  1056,  2022, 11004,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [  101,  1037,  1059, 21030,  4371,  2003,  1037,  2152,  8219,
          3315, 13896, 25090,  3560, 11192,  2614,  2550,  2011,  2250,
         12314,  2083,  2019, 19470,  2135,  8061,  2030, 16620, 13095,
          2008,  2788,  5158,  2083,  2019,  2330,  6357,  2030,  2019,
          4722, 27208,  1999,  1996, 11192, 17790,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [  101,  1996,  1059, 21030,  43

## Training

In [16]:
model = ContextEncoderComplex(final_dropout=0.8,
                               dense_neurons=256,
                             lstm_size=256,
                             lstm_dropout_percentage=0.75,
                             cnn_filters=8,
                             cnn_kernel_size=3,
                             pool_size=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [17]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy')
]

In [18]:
EPOCHS = 1
BATCH_SIZE = 5

# balanced = balanced binary crossentropy
checkpoint_filepath = '../models/LDABERT/complex/{}-{}-{}-pct-{}-aug/checkpoint.ckpt'.format(
                        dataset.dataset_type,                    
                        len(sentences), 
                        dataset.pct_data,
                        dataset.augment_pct)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=False,
    mode="auto",
    save_freq="epoch")

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

callbacks = [
#     early_stopping,
    model_checkpoint_callback
]

In [19]:
checkpoint_filepath

'../models/LDABERT/complex/clinical-10-0.001-pct-0.001-aug/checkpoint.ckpt'

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=METRICS)

In [21]:
try:
    model.load_weights(checkpoint_filepath)
except:
    print("No checkpoint available.")

In [22]:
# remove warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
history = model.fit([
                        left_input, mid_input, right_input, 
                        lda_left_input, lda_mid_input, lda_right_input
                    ], 
                    tf.convert_to_tensor(saved_labels), 
                    epochs=EPOCHS,
                    validation_split=0.25,
                    batch_size=BATCH_SIZE,
                    verbose=1, 
#                         class_weight=class_weight,
                    callbacks=callbacks)

UnimplementedError:  Cast string to float is not supported
	 [[node binary_crossentropy/Cast (defined at <ipython-input-22-58df2e6e0a4a>:3) ]] [Op:__inference_train_function_49855]

Function call stack:
train_function


In [ ]:
model.summary()

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()